# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [181]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [182]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [147]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [173]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [149]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Identify types of emotions

In [150]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, impressed, grateful, pleased


## Identify anger

In [151]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [152]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [153]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## Inferring topics

In [154]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [155]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement


In [160]:
response.split(sep=',')

['[1', ' 0', ' 0', ' 1', ' 1]']

In [161]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [164]:
# JSON version
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as json object with the key as the topic name and the key as 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
  "nasa": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 1,
  "federal government": 1
}


In [170]:
# List version
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer with the format "topic name: 0 or 1" for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [171]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Try experimenting on your own!

In [178]:
# Review Text
gaming_review = """
I've been on the hunt for a classic retro game handheld console, \
something that brings back those good old gaming memories but \
with modern touches. After a bit of research, I stumbled upon \
the RetroPlay handheld by PixelJoy. It wasn't too pricey, and \
the feature set was impressive for the cost. The delivery was \
swift, which was a pleasant surprise.

However, when it arrived, I noticed a small issue with the \
screen - it had a minor scratch. I reached out to PixelJoy's \
customer service, and they were incredibly accommodating. \
They sent a replacement screen, which I received in just a \
few days. Installing the new screen was a breeze, thanks to \
the clear instructions provided.

One thing I really appreciated was the extensive game library \
pre-loaded on the device. It's like a trip down memory lane, \
with all my favorite classics in one place. The console itself \
feels sturdy and well-made, with responsive buttons and a crisp \
display that does justice to the retro games.

I had a minor hiccup when I realized the package didn't include \
a charging cable. Contacting PixelJoy's support was a smooth \
process, and they promptly sent the missing cable.

Overall, I'm impressed with PixelJoy as a brand. They genuinely \
care about their customers' satisfaction and are quick to resolve \
any issues. The RetroPlay console has become my go-to for gaming \
nostalgia, and I highly recommend it to anyone looking for a quality \
retro gaming experience.
"""

In [183]:
# Sentiment Analysis
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.
positive


In [184]:
# Identify types of emotion
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

impressed, appreciative, satisfied, nostalgic, recommend


In [185]:
# Identify anger
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

No


In [186]:
# Extract product and company name from customer reviews
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "RetroPlay handheld",
  "Brand": "PixelJoy"
}


In [187]:
# Doing multiple tasks at once
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "RetroPlay handheld",
  "Brand": "PixelJoy"
}


In [188]:
# Inferring topics
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

1. Retro game handheld console
2. PixelJoy
3. Customer service
4. Game library
5. Charging cable


In [194]:
# Make a news alert for certain topics

topic_list = [
    "Retro game handheld console", "Game library", "Screen Display", 
    "Charging cable", "Gameplay"
]


prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer with the format "topic name: 0 or 1" for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{gaming_review}'''
"""
response = get_completion(prompt)
print(response)

Retro game handheld console: 1
Game library: 1
Screen Display: 1
Charging cable: 1
Gameplay: 0


In [198]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['Retro game handheld console'] == 1:
    print("ALERT: New Retro game handheld console story!")

ALERT: New Retro game handheld console story!
